<a href="https://colab.research.google.com/github/joepareti54/joepareti54/blob/main/jp_llama_langchain_rag_faiss_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install replicate
!pip install langchain
!pip install faiss-cpu
#
!pip install sentence-transformers

In [2]:
print('test llama langchain RAG FAISS using kb emebddings')

test llama langchain RAG FAISS using kb emebddings


In [3]:
import os
from langchain.llms import Replicate

# Set your Replicate API token in the environment variables
os.environ['REPLICATE_API_TOKEN'] = "r8_PukHyR1PKKUzCJpyQdodF4ZWdqXVrCM0uBiLt"
replicate_api_token = os.getenv('REPLICATE_API_TOKEN')

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain, LLMChain
from langchain.chains.sequential import SequentialChain
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

In [5]:
# Initialize the Replicate LLaMA model
llama_model = Replicate(
    model="meta/llama-2-7b-chat:f1d50bb24186c52daae319ca8366e53debdaa9e0ae7ff976e918df752732ccc4",
    model_kwargs={"temperature": 0.01, "max_length": 500, "top_p": 1}
)

In [6]:
# Initialize SentenceTransformer Model for Embedding Generation
model = SentenceTransformer('all-MiniLM-L6-v2')
knowledge_base = {
    "space exploration": "Space exploration involves the use of astronomy and space technologies to explore outer space. While the study of space is carried out mainly by astronomers with telescopes, its physical exploration is conducted both by unmanned robotic space probes and human spaceflight.",
    "mars": "Mars is the fourth planet from the Sun and the second-smallest planet in the Solar System, being larger than only Mercury. In English, Mars carries the name of the Roman god of war and is often referred to as the 'Red Planet' because the iron oxide prevalent on its surface gives it a reddish appearance.",
    "international space station": "The International Space Station (ISS) is a space environment research laboratory that also acts as a habitat for a crew of astronauts and cosmonauts. The station is a joint project between five participating space agencies: NASA (United States), Roscosmos (Russia), JAXA (Japan), ESA (Europe), and CSA (Canada).",
    "black holes": "A black hole is a place in space where gravity pulls so much that even light cannot get out. The gravity is so strong because matter has been squeezed into a tiny space. This can happen when a star is dying. Because no light can get out, people can't see black holes.",
    "the big bang theory": "The Big Bang theory is the prevailing cosmological model for the observable universe from the earliest known periods through its subsequent large-scale evolution. The model describes how the universe expanded from a very high-density and high-temperature state and offers a comprehensive explanation for a broad range of phenomena, including the abundance of light elements, the cosmic microwave background radiation, and large-scale structure.",
    "exoplanets": "Exoplanets, or extrasolar planets, are planets that orbit a star other than the Sun. Over 4,000 exoplanets have been discovered in the Milky Way Galaxy as of 2020. These discoveries were made using a variety of detection methods, such as the transit method, radial velocity method, and direct imaging.",
}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
# Generate Embeddings for Knowledge Base Entries
kb_entries = list(knowledge_base.values())
kb_embeddings = model.encode(kb_entries)  # Generate embeddings
kb_embeddings = np.array(kb_embeddings).astype('float32')  # Ensure correct type for FAISS

# Create and Train a FAISS Index
index = faiss.IndexFlatL2(kb_embeddings.shape[1])  # L2 distance for similarity
index.add(kb_embeddings)  # Add embeddings to index

In [8]:
def retrieve_knowledge(topic):
    # Encode the input topic to a vector using the same model
    topic_embedding = model.encode([topic])[0].astype('float32')

    # Use FAISS to find the most relevant knowledge base entry
    D, I = index.search(np.array([topic_embedding]), 1)  # Search for the top 1 closest entry
    most_relevant_entry = kb_entries[I[0][0]]

    return most_relevant_entry

In [9]:
llm = llama_model

# Setup for generating a question and then answering it remains the same
generate_question_template = PromptTemplate(
    input_variables=["input"],
    template="Generate a thoughtful question about {input}."
)

In [10]:
# LLMChain for generating a question
generate_question_chain = LLMChain(llm=llm, prompt=generate_question_template)

# Function to combine knowledge retrieval and answer generation
def generate_answer_with_knowledge(topic, question):
    """Combines retrieved knowledge with the question to generate an enriched answer."""
    knowledge = retrieve_knowledge(topic)
    combined_input = f"{question} Based on the following knowledge: {knowledge}"
    # Directly use the llm instance to generate an answer based on the combined input
#    answer = llm.run(prompt=combined_input)  # Adjust based on the correct method
    answer = llm(prompt=combined_input)  # Adjust based on the correct method
    return answer

# Function to run the enhanced sequence
def run_chain(topic):
    # Generate a question based on the topic
    question_result = generate_question_chain.run(input={"input": topic})
    print(f"Question: {question_result}")

    # Generate an answer using the question and retrieved knowledge
    answer_result = generate_answer_with_knowledge(topic, question_result)
    print(f"Answer: {answer_result}")


In [11]:
# Example topic to generate a question and answer
topic = "what are exoplanets and can we get there"
topic = "How do black holes form, and why can't we see them with our eyes or telescopes?"
run_chain(topic)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Question:  Great question! Black holes are fascinating objects in space that have captivated the imagination of scientists and the public alike. Here's a thoughtful question related to your input:
"What are the different theories on how black holes form, and how do they affect our understanding of the universe's evolution and structure? Additionally, how do scientists use various telescopes and observation methods to study black holes without directly observing them, and what are some of the challenges and limitations of these methods?"
This question allows for a deeper exploration of the complex and intriguing topic of black holes, while also highlighting the importance of scientific inquiry and the need for continued research and discovery. By examining the different theories on black hole formation and the methods used to study them, we can gain a better understanding of the universe's mysteries and the ways in which science can help us uncover them.


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Answer:  Great question! Black holes are indeed fascinating objects in space that have captivated the imagination of scientists and the public alike. There are several theories on how black holes form, and these theories have significant implications for our understanding of the universe's evolution and structure.
One of the most widely accepted theories on black hole formation is the collapse theory. According to this theory, a massive star collapses under its own gravity, causing the matter to be compressed into an incredibly small space. This compression creates an intense gravitational field that pulls everything, including light, towards the center of the black hole.
Another theory is the merger theory, which suggests that black holes form when two neutron stars or a neutron star and a black hole collide. This collision causes the matter to be compressed and heated, resulting in the formation of a new black hole.
There are also other theories, such as the primordial black hole the